In [371]:
import pandas as pd
import numpy as np
import os
import pickle

from config import BASE_DIR
print(os.getcwd())
os.chdir(BASE_DIR)
from scripts.backtesting.backtesting import Strategy, Backtest 
from scripts.backtesting.lib import resample_apply, plot_heatmaps
from ta.trend import EMAIndicator
import scripts.functions as fn 


C:\Users\lbisa\Dropbox\Cripto\python\InvestingEvolution\ie\www


In [372]:
#klines_file = './backtest/klines/2d01/Completo_BTCUSDT_2d01_2021-01-01_2023-07-31.DataFrame'
#klines_file = './backtest/klines/0m15/Completo_XRPUSDT_0m15_2021-01-01_2023-07-31.DataFrame'
#klines_file  = './backtest/klines/0m01/Completo_BTCUSDT_0m01_2021-01-01_2023-07-31.DataFrame'
#klines_file = './backtest/klines/0m01/Lateral_BTCUSDT_0m01_2023-06-20_2023-10-09.DataFrame'

#klines_file  = './backtest/klines/1h01/Completo_BTCUSDT_1h01_2021-01-01_2023-07-31.DataFrame'
#klines_file  = './backtest/klines/1h01/Alcista_BTCUSDT_1h01_2020-09-07_2021-05-03.DataFrame'
#klines_file = './backtest/klines/1h01/Lateral_BTCUSDT_1h01_2023-06-20_2023-10-09.DataFrame'
klines_file = './backtest/klines/1h01/Bajista_BTCUSDT_1h01_2021-04-12_2021-07-12.DataFrame'

#klines_file  = './backtest/klines/1h04/Completo_BTCUSDT_1h04_2021-01-01_2023-07-31.DataFrame'

with open(klines_file, 'rb') as file:
    data = pickle.load(file)
data.set_index('datetime',inplace=True)
data.rename(columns={'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'volume': 'Volume'}, inplace=True)
#data['sma'] = data['Close'].rolling(21).mean()

start_date = klines_file.split('/')[4].split('_')[3] #Extrae la fecha de inicio
start_date = pd.to_datetime(start_date, format='%Y-%m-%d')
start_date = start_date.replace(hour=0, minute=0, second=0)
data = data[data.index >= start_date]




In [373]:
def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

   



In [374]:


class SmaCross(Strategy):
    
    # for later optimization
    ma = 14         #Periodos para Media movil simple 
    quote_perc = 90 #% de compra inicial, para stock
    re_buy_perc = 4 #% para recompra luego de una venta
    lot_to_safe = 5 #% a resguardar si supera start_cash


    #Resultados de la optimizacion
    #Bajista   ma=14,quote_perc=90,re_buy_perc=4,lot_to_safe=10
    #Lateral   ma=14,quote_perc=90,re_buy_perc=10,lot_to_safe=6
    #Alcista   ma=28,quote_perc=90,re_buy_perc=2,lot_to_safe=10
    #Completo  ma=14,quote_perc=90,re_buy_perc=4,lot_to_safe=2

    """
    General   ma=14,quote_perc=90,re_buy_perc=4,lot_to_safe=5
    Completo:   Return (Ann.) [%] 15.00  - Max. Drawdown [%] -41.24
    Alcista:    Return (Ann.) [%] 364.21 - Max. Drawdown [%] -11.70
    Lateral:    Return (Ann.) [%] -12.59 - Max. Drawdown [%] -14.62
    Bajista:    Return (Ann.) [%] -27.55 - Max. Drawdown [%] -21.91
    """


    op_last_price = 0
    
    start_cash = 0.0  #Cash correspondiente a la compra inicial
    pre_start = False #Controla que en la primera vela se compren la sunidades para stock 

    def init(self):
        super().init()
        self.sma = self.I(SMA, self.data.Close, self.ma)
        
        

       
        
    def next(self):
        
        price = self._broker.last_price
        avg_price = (self.data.High[-1]+self.data.Low[-1])/2

        #Compra inicial para estockearse de Monedas
        if not self.pre_start:
            size = (self._broker._cash * (self.quote_perc/100)) / price
            self.buy(size=size)
            self.start_cash = price*size
            self.pre_start = True 

        #Estrategia
        else:
            hold = self._broker.position.size*price

            if avg_price > self.sma[-1] and hold > self.start_cash*(1+(self.lot_to_safe/100)):
                size = (hold - self.start_cash)/price
                limit_price = price*(1-(self.re_buy_perc/100))
                if size*price > 12 and size*limit_price: #Intenta recomprar solo si la compra es por las de 12 dolares
                    self.sell(size=size)
                    self.buy(size=size,limit=limit_price)

            elif avg_price < self.sma[-1] and hold < self.start_cash*(1-(self.lot_to_safe/100)):
                size = self._broker.position.size*(self.lot_to_safe/100)
                limit_price = price*(1-(self.re_buy_perc/100))
                if size*price > 12 and size*limit_price: #Intenta recomprar solo si la compra es por las de 12 dolares
                    self.op_last_price = avg_price
                    self.sell(size=size)
                    self.buy(size=size,limit=limit_price)

            



In [375]:
#Quita las primeras 210 velas, que son agregadas al descargar datos para preveer el armado de indicadores 
#y que la estrategia inicie con los indicadores previamente armados 


cash = 1000
bot = Backtest(data, SmaCross, cash=cash, commission=.002, trade_on_close=False)
stats = bot.run()

print(f'Periodo: {stats.Start} al {stats.End}')
print(f'Duracion: {stats.Duration}')
print('Return (Ann.) [%]',stats['Return (Ann.) [%]'])
print('Max. Drawdown [%]',stats['Max. Drawdown [%]'])
print('# Trades',stats['# Trades'])
trades = stats._trades.copy()


bot.plot(open_browser=True, plot_trades=False, resample='1D')

Periodo: 2021-04-12 00:00:00 al 2021-07-12 23:00:00
Duracion: 91 days 23:00:00
Return (Ann.) [%] -27.555632744731074
Max. Drawdown [%] -21.918652030047035
# Trades 902


GridPlot(id='p50577', ...)

In [376]:
#Optimizacion
"""
parametros
quote_perc = 0.75  #% de compra inicial, para stock
re_buy_perc = 0.08 #% para recompra luego de una venta
lot_to_safe = 0.04 #% a resguardar si supera start_cash
"""

def check_optimize(series):
    result = series['Max. Drawdown [%]']+10*series['Return (Ann.) [%]']
    return result

optimizar = False
if optimizar:
    stats = bot.optimize(
        #ma = [14,21,28],
        #quote_perc = [30,60,90],
        re_buy_perc = [2,4,6,8,10],
        lot_to_safe = [2,4,6,8,10],
        maximize = check_optimize,

    )

    print(stats._strategy)
    print(f'Periodo: {stats.Start} al {stats.End}')
    print(f'Duracion: {stats.Duration}')
    print('Return (Ann.) [%]',stats['Return (Ann.) [%]'])
    print('Max. Drawdown [%]',stats['Max. Drawdown [%]'])
    print('# Trades',stats['# Trades'])

